In [20]:
%reload_ext nb_black

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [28]:
import pandas as pd
import numpy as np

from tqdm.notebook import tqdm

from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans, MeanShift

import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

data_url = "https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/Data%20Sets%20Clustering/starbucks_locations.csv"
starbs = pd.read_csv(data_url)
starbs.shape

(25600, 13)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [22]:
starbs.head()

,Brand,Store Number,Store Name,Ownership Type,Street Address,City,State/Province,Country,Postcode,Phone Number,Timezone,Longitude,Latitude
0,Starbucks,47370-257954,"Meritxell, 96",Licensed,"Av. Meritxell, 96",Andorra la Vella,7,AD,AD500,376818720,GMT+1:00 Europe/Andorra,1.53,42.51
1,Starbucks,22331-212325,Ajman Drive Thru,Licensed,"1 Street 69, Al Jarf",Ajman,AJ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,55.47,25.42
2,Starbucks,47089-256771,Dana Mall,Licensed,Sheikh Khalifa Bin Zayed St.,Ajman,AJ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,55.47,25.39
3,Starbucks,22126-218024,Twofour 54,Licensed,Al Salam Street,Abu Dhabi,AZ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,54.38,24.48
4,Starbucks,17127-178586,Al Ain Tower,Licensed,"Khaldiya Area, Abu Dhabi Island",Abu Dhabi,AZ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,54.54,24.51


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Filter to East TN using the below latitude and longitude bounds.

In [23]:
lat_min = 35.71
lat_max = 36.55
lon_min = -84.55
lon_max = -82.37

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [31]:
starbs = starbs[(starbs["Latitude"] >= lat_min) & (starbs["Latitude"] <= lat_max)]
starbs = starbs[(starbs["Longitude"] >= lon_min) & (starbs["Longitude"] <= lon_max)]
starbs.shape

(48, 14)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Create a plot of the East TN Starbucks location and include a point on the map that shows where Adam lives.

The plot code is already written for you, your job is to create a `plot_df` that will make the plot code work.

In [33]:
adam_lat = 36.31
adam_lon = -82.35

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [36]:
# Define plot_df
adam_row = {"Latitude": adam_lat, "Longitude": adam_lon, "Label": "Adam"}

plot_df = starbs[["Latitude", "Longitude"]].copy()
plot_df["Label"] = "Starbucks"

plot_df = plot_df.append(adam_row, ignore_index=True)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [64]:
px.scatter_geo(plot_df, "Latitude", "Longitude", color="Label", scope="usa")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Gaussian mixture

Isolate just the latitude and longitude columns.

In [39]:
lat_lon = starbs[["Latitude", "Longitude"]].copy()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [40]:
k = 5

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

* Create and fit a `KMeans()` model

In [41]:
kmeans = KMeans(k)
kmeans.fit(lat_lon)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=5, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

* Create and fit a `GaussianMixture()` model

In [42]:
gauss = GaussianMixture(k)
gauss.fit(lat_lon)

GaussianMixture(covariance_type='full', init_params='kmeans', max_iter=100,
                means_init=None, n_components=5, n_init=1, precisions_init=None,
                random_state=None, reg_covar=1e-06, tol=0.001, verbose=0,
                verbose_interval=10, warm_start=False, weights_init=None)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

* Use the `GaussianMixture.predict_proba()` method.
* What are the top 3 observations we are least confident about?
* What are the top 3 observations we are most confident about?

In [69]:
prob_df = pd.DataFrame(gauss.predict_proba(lat_lon), index=lat_lon.index)
prob_df["max_prob"] = prob_df.max(axis=1)
prob_df["label"] = gauss.predict(lat_lon)

print("Top 3 iffiest points")
display(prob_df.sort_values("max_prob").head(3))

print("Top 3 confident points")
display(prob_df.sort_values("max_prob").tail(3))

Top 3 iffiest points


,0,1,2,3,4,max_prob,label
22952,0.0,0.384645,1.936237e-48,0.0,0.615355,0.615355,4
22937,0.0,0.384645,1.936237e-48,0.0,0.615355,0.615355,4
22895,0.0,0.695265,3.791555e-44,0.0,0.304735,0.695265,1


Top 3 confident points


,0,1,2,3,4,max_prob,label
22934,1.0,2.276323e-109,1.523467e-67,0.0,4.867245e-166,1.0,0
22927,1.0,2.121407e-127,3.082372e-111,0.0,2.613756e-177,1.0,0
22928,1.0,5.985983e-126,5.908057e-110,0.0,3.791965e-175,1.0,0


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Replot the data twice:
1. Color points by their kmeans cluster labels
2. Color points by their gauss cluster labels

You can use the below code to crop the map plot to our desired area:
```python
fig = px.scatter_geo(
    # Awesome plot code here
)

fig.update_layout(geo={'lataxis': {'range': [lat_min - .1, lat_max + .1]}, 
                       'lonaxis': {'range': [lon_min - .1, lon_max + .1]}})
fig.show()
```

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [80]:
gauss_plot = lat_lon.copy()
gauss_plot["label"] = gauss.predict(lat_lon)
gauss_plot["size"] = 1 - gauss.predict_proba(lat_lon).max(axis=1) + 0.1

centers_df = pd.DataFrame(gauss.means_, columns=["Latitude", "Longitude"])
centers_df["label"] = "Cluster center"

gauss_plot = pd.concat((gauss_plot, centers_df), sort=False)

fig = px.scatter_geo(
    gauss_plot,
    "Latitude",
    "Longitude",
    color="label",
    #     size="size",
    hover_name=gauss_plot.index,
)

fig.update_layout(
    geo={
        "lataxis": {"range": [lat_min - 0.1, lat_max + 0.1]},
        "lonaxis": {"range": [lon_min - 0.1, lon_max + 0.1]},
    }
)
fig.show()

# ----------------------------------------------------------------------------

kmeans_plot = lat_lon.copy()
kmeans_plot["label"] = kmeans.labels_

centers_df = pd.DataFrame(kmeans.cluster_centers_, columns=["Latitude", "Longitude"])
centers_df["label"] = "Cluster center"

kmeans_plot = pd.concat((kmeans_plot, centers_df), sort=False)

fig = px.scatter_geo(
    kmeans_plot, "Latitude", "Longitude", color="label", hover_name=kmeans_plot.index,
)

fig.update_layout(
    geo={
        "lataxis": {"range": [lat_min - 0.1, lat_max + 0.1]},
        "lonaxis": {"range": [lon_min - 0.1, lon_max + 0.1]},
    }
)
fig.show()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [77]:
kmeans.cluster_centers_

array([[ 36.        , -83.95909091],
       [ 36.37125   , -82.42125   ],
       [ 35.87166667, -83.52833333],
       [ 35.87888889, -84.06944444],
       [ 35.906     , -84.32      ]])

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [76]:
gauss.means_

array([[ 36.37125   , -82.42125   ],
       [ 35.9283612 , -84.01371155],
       [ 35.87166667, -83.52833333],
       [ 35.89      , -84.55      ],
       [ 35.90400454, -84.20720279]])

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Mean-shift

* Use `matplotlib` to plot the East TN Starbucks
* Label the `x` and `y` axes

Use `sns.kdeplot()` to show the distribution of Latitude and Longitude separately.

* Use `sns.kdeplot()` to show the 2d distribution of Latitude and Longitude.
* Add the starbucks locations to the plot

Use `MeanShift()` to cluster the observations

Redo the above plot colored by cluster label